## Bibliotecas

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import requests
import re
from math import radians
import time
from haversine import haversine
import pandas as pd
from pandas import json_normalize
from geopy.geocoders import Nominatim
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
#from bs4 import BeautifulSoup
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
import sys
sys.setrecursionlimit(2000)

# Show all Columns and Rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# bliblioteca = https://pypi.org/project/haversine/

print('Libraries imported')

Libraries imported


## Abrir o arquivo

In [2]:
df_estab = pd.read_csv('2_not_agrup_trat.csv')
df_estab.drop(df_estab.columns[[0]],axis=1, inplace=True)

In [3]:
df_estab.head(2)

,name,endereço,estabelecimento,latitude,longitude,estacao_nome,estacao_lat,estacao_lon,distancia
0,Brasas,"Rua Pompeu Loureiro, 41, Rio de Janeiro, RJ, 2...",educação,-22.973412,-43.19220,General Osório,-22.984611,-43.197183,1345
1,Academia do Tap,"Rua Visconde de Pirajá, 235, Rio de Janeiro, R...",academia_ginasio_estudio,-22.984525,-43.20231,General Osório,-22.984611,-43.197183,524


In [4]:
df_padarias = df_estab[df_estab['estabelecimento'] == 'padaria']

In [5]:
df_padarias.head(2)

,name,endereço,estabelecimento,latitude,longitude,estacao_nome,estacao_lat,estacao_lon,distancia
2,La Veronese,"Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",padaria,-22.984827,-43.19636,General Osório,-22.984611,-43.197183,87
6,Padaria Santa Marta,"Avenida Epitácio Pessoa, 1789, Rio de Janeiro, RJ",padaria,-22.978994,-43.20080,General Osório,-22.984611,-43.197183,726


In [6]:
df_padarias.shape

(199, 9)

In [7]:
# Chaves para acesso:

client_id = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'
client_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
oauth_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'
radius = 50
limit = 50
# categories
# latitude
# longitude

In [8]:
def make_request(url):
  headers = {
      "Accept": "application/json",
      "Authorization": "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
  }
  response = requests.request("GET", url, headers=headers)
  return response

def get_next_url(response):
  try:
    return re.findall("<(.*?)>", response.headers["Link"])[0]
  except (KeyError, IndexError):
    return None

def get_nearby_venues_data(station_dict):
  json_list = []
  url = f"https://api.foursquare.com/v3/places/search?ll={station_dict}&radius=500&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50"

  while url != None:
    response = make_request(url)
    json_file = response.json()
    json_list.append(json_file)
    
    url = get_next_url(response)
    time.sleep(0.1)
    print(url)
    print(response.headers)

  return json_list

In [9]:
dict_places = {}
for latlon in df_padarias[['name', 'latitude', 'longitude']].values:
  response = get_nearby_venues_data(f'{latlon[1]},{latlon[2]}')
  df_response = pd.DataFrame.from_dict(response[0]['results'])
  dict_categories = {}
  for category in df_response['categories']:
    for item in category:
      try:
        dict_categories[item['name']] += 1
      except:
        dict_categories[item['name']] = 1
  dict_places[latlon[0]] = dict_categories

  dict_id = {}
  for id in df_response['categories']:
    for item in id:
      try:
        dict_id[item['id']] += 1
      except:
        dict_id[item['id']] = 1
  dict_places[latlon[0]] = dict_id 

https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=500&ll=-22.984827%2C-43.19636&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50
{'Connection': 'keep-alive', 'Content-Length': '4428', 'Content-Type': 'application/json; charset=utf-8', 'Link': '<https://api.foursquare.com/v3/places/search?cursor=c3I6NTA&radius=500&ll=-22.984827%2C-43.19636&fields=fsq_id%2Cname%2Cgeocodes%2Clocation%2Ccategories&limit=50>; rel="next"', 'Server': 'Finatra', 'X-Fsq-Request-ID': '62703746d8319606a519e0a8', 'X-RateLimit-Path': '/v3/places/search', 'X-RateLimit-Limit': '0', 'X-RateLimit-Remaining': '0', 'content-encoding': 'gzip', 'Accept-Ranges': 'bytes', 'Date': 'Mon, 02 May 2022 19:55:50 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-gig2250061-GIG', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1651521350.555364,VS0,VE901', 'Vary': 'Origin'}
https://api.foursquare.com/v3/places/search?cursor=c3I6MTAw&radius=500&ll=-22.984827%2C-43.19636&fields=fsq_id%2Cname%2Cgeoc

In [10]:
df_places =  pd.DataFrame.from_dict(dict_places).T.fillna(0)
df_places.head(2)

,13002,13064,13236,13039,13325,13027,17067,17071,17065,13032,13065,13338,13003,13079,13309,11061,11045,13030,16041,13310,13034,13145,13006,13142,17069,10052,17035,13083,13383,17003,16045,13011,17134,13021,16003,13297,10032,13013,13016,13302,16001,11073,10000,16037,10028,16018,17043,13054,13007,11065,17048,13018,13068,13035,13009,17045,13263,17044,13072,17052,10039,17117,18000,13148,17047,13046,13332,13347,13322,13177,11064,17082,17030,13010,10043,17018,17000,17110,11062,17089,17062,13303,17084,11075,18025,13028,17131,13026,13040,13377,10003,13031,13053,13089,13025,13342,11067,10027,13276,10024,11071,16032,16004,10047,11148,12094,13099,10049,19014,13055,11068,17029,17115,13049,13080,17059,17113,16026,10015,13047,10030,18075,16049,13005,16000,13059,13057,13036,13311,13382,17068,16053,13001,11056,13000,13022,17054,13087,13334,18065,17028,10051,17119,13048,17114,11097,17100,13097,11137,17060,10020,10004,17107,13134,13086,13070,13343,13387,13090,15011,11139,13139,13324,17137,11000,10026,17002,17077,13286,13354,17080,17042,11003,15051,17138,17006,17096,18039,13029,17039,17102,13067,16027,17041,13345,16040,16046,17019,18048,17061,17064,13041,17005,10037,13058,17070,11041,17025,11001,11009,17010,11010,13165,10045,17033,11107,18064,16044,11152,12013,17112,17046,17058,12063,13044,11030,18035,15000,10038,16030,18018,13352,16043,10055,13015,16019,11038,17012,17090,13084,11069,13298,17076,18047,12082,19007,16033,19010,19013,16017,12005,18056,18066,16020,12000,11117,10036,10016,18055,13051,18061,11036,18021,10009,11138,12009,10006,13199
La Veronese,3.0,5.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,7.0,2.0,6.0,4.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Padaria Santa Marta,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_places.to_csv("3_not_df_cluster_dummie.csv")